In [ ]:
import sympy as sym
sym.init_printing()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import InterpolatedUnivariateSpline

In [ ]:
plt.style.use('./my_style.py')
import matplotlib.ticker as mticker

from cycler import cycler
import matplotlib.cm     as cm

# Example use of solver 

In [ ]:
from solver import KmouExpansionJordan

In [ ]:
A0=1

In [ ]:
# initilise solver
dum_kmou = KmouExpansionJordan(beta=0.1, K0=1, lamb_val=10, Om0=0.3089, A0=A0)

In [ ]:
# dum_kmou.tune_lambda(E_target=1.105861, a_target=0.875543)
dum_kmou.tune_lambda(E_target=1, a_target=1, frame='Jordan', maxiter=10) # tune lambda to recover E=E_target in the desired frame
# dum_kmou.solve() # to compute solutions without tuning

In [ ]:
dum_kmou.lamb_val

In [ ]:
# create dataframe with some output quantities
header, table = dum_kmou.eval()
df_out = pd.DataFrame(np.array(table).T, columns=header).set_index('a')

In [ ]:
print(A0)
A0 = df_out.loc[1,'A']
A0

In [ ]:
# Check E
plt.semilogx(df_out.E_kmou/(0.3089*(df_out.index**(-3)-1)+1)**(1/2))
# plt.hlines(-3/2, 1e-3, 1, colors='k', linestyles='--'

In [ ]:
# Check E
plt.plot(1/df_out.index-1, df_out.E_kmou/(0.3089*(df_out.index**(-3)-1)+1)**(1/2))
plt.xlim(0,6)

In [ ]:
# Check E'/E
plt.semilogx(df_out.E_kmou_a*df_out.index/df_out.E_kmou)
plt.hlines(-3/2, 1e-3, 1, colors='k', linestyles='--')

### Comparison with Ben: $K_0=1$, $\beta=0.2$, $n=2$

In [ ]:
header_str = 'a,  H_MG,  H_LCDM, H_MG/H_LCDM,  aH dH/da / H0^2 ,  phi , d phi/ dlna'
header = [s.strip(' ')for s in header_str.split(',')]

df_Ben_BG = pd.DataFrame(np.loadtxt('./Data/Background/background_k01_glam.dat'), columns=header).set_index('a')

In [ ]:
df_Ben_BG['A'] = np.e**(df_Ben_BG.phi*0.2)
df_Ben_BG['a_J'] = df_Ben_BG['A']*df_Ben_BG.index

In [ ]:
a_E = df_Ben_BG.index#np.logspace(-4,0,1000)
a_J = dum_kmou.get_a_Jor(a_E)

In [ ]:
header, table = dum_kmou.eval(a_vals=a_J)
df_out = pd.DataFrame(np.array(table).T, columns=header).set_index('a')

In [ ]:
dum_kmou.get_E_Ein(a_Ein=a_E)

In [ ]:
df_out.index * 0.1* df_out['phi_a']

In [ ]:
plt.plot(1/a_J-1, (dum_kmou.get_E_Ein(a_Ein=a_E)/df_out['E_kmou']))
plt.plot(1/a_J-1, df_out.A*(1- df_out.index * 0.1* df_out['phi_a']), '--')
plt.xlim(0,6)

In [ ]:
plt.semilogx((df_out['E_kmou']/dum_kmou.E_LCDM_fun(a_J)))
plt.semilogx(df_Ben_BG.a_J,(df_Ben_BG['H_MG']/df_Ben_BG['H_LCDM']),'--')
# plt.ylim(0.9,1.1)

# Compute growth factor

In [ ]:
dum_kmou.get_growth()

In [ ]:
dum_LCMD = KmouExpansionJordan()
dum_LCMD.solve()
dum_LCMD.get_growth_LCDM()

In [ ]:
plt.plot(dum_kmou.D_kmou_J['t'],dum_kmou.D_kmou_J['y'][1])
plt.plot(dum_LCMD.D_LCDM['t'],dum_LCMD.D_LCDM['y'][1])


In [ ]:
# produce familiar growth ratio with LCDM
a_E = np.logspace(-4,0,1000)
a_J = a_E#dum_kmou.get_a_Jor(a_E)

plt.semilogx(a_E , (dum_kmou.D_kmou_J.sol(a_J)[1]/dum_LCMD.D_LCDM.sol(a_E)[1]))


In [ ]:
(dum_kmou.D_kmou_J.sol(a_J)[1]/dum_LCMD.D_LCDM.sol(a_E)[1])**2